## 第一部分 - 人脸识别

人脸识别系统通常被分为两大类：

- **人脸验证：**“这是不是本人呢？”，比如说，在某些机场你能够让系统扫描您的面部并验证您是否为本人从而使得您免人工检票通过海关，又或者某些手机能够使用人脸解锁功能。这些都是1：1匹配问题。

- **人脸识别：**“这个人是谁？”，比如说，在视频中的百度员工进入办公室时的脸部识别视频的介绍，无需使用另外的ID卡。这个是1：K的匹配问题。

FaceNet可以将人脸图像编码为一个128位数字的向量从而进行学习，通过比较两个这样的向量，那么我们就可以确定这两张图片是否是属于同一个人。

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, MaxPooling2D, AveragePooling2D, Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.python.keras.engine import Layer

In [5]:
K.set_image_data_format('channels_first')

import time
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import fr_utils
from inception_blocks_v2 import *

### 1 - 将人脸图像编码为128位的向量
#### 1.1 - 使用卷积网络来进行编码

In [6]:
# 获取模型
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

#打印模型的总参数数量
print("参数数量： {}".format(FRmodel.count_params()))

参数数量： 3743280


通过使用128神经元全连接层作为最后一层，
该模型确保输出是大小为128的编码向量，然后比较两个人脸图像的编码

因此，如果满足下面两个条件的话，编码是一个比较好的方法：

- 同一个人的两个图像的编码非常相似。
- 两个不同人物的图像的编码非常不同。

#### 1.2 - 三元组损失函数

In [7]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    """
    实现三元组损失函数
    Args:
        y_true: true标签，当你在Keras里定义了一个损失函数的时候需要它，但是这里不需要。
        y_pred:列表类型，包含了如下参数：
            anchor -- 给定的“anchor”图像的编码，维度为(None,128)
            positive -- “positive”图像的编码，维度为(None,128)
            negative -- “negative”图像的编码，维度为(None,128)

        alpha: 超参数，阈值

    Returns:
            loss -- 实数，损失的值
    """
        #获取anchor, positive, negative的图像编码
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)

    #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)

    #第三步：减去之前的两个距离，然后加上alpha
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)

    #通过取带零的最大值和对训练样本的求和来计算整个公式
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))

    return loss

In [8]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed=1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed= 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed=1))
    loss = triplet_loss(y_true, y_pred)

    print("loss = {}".format(loss.eval()))

loss = 528.142578125


### 2 - 加载训练好了的模型

In [9]:
#开始时间
start_time = time.perf_counter()

#编译模型
FRmodel.compile(optimizer='adam', loss=triplet_loss, metrics=['accuracy'])

#加载权值
fr_utils.load_weights_from_FaceNet(FRmodel)

#结束时间
end_time = time.perf_counter()

#计算时差
minium = end_time - start_time

print("执行了：{}分{}秒".format(int(minium / 60), int(minium % 60)))

执行了：1分23秒


### 3 - 模型的应用
#### 3.1 - 人脸验证

我们构建一个数据库，里面包含了允许进入的人员的编码向量，我
们使用`fr_uitls.img_to_encoding(image_path, model)`函数来生成编码，它会根据图像来进行模型的前向传播。

我们这里的数据库使用的是一个字典来表示，这个字典将每个人的名字映射到他们面部的128维编码。

In [10]:
database = {}
persons = ['danielle', 'younes', 'tian',
           'andrew', 'kian', 'dan',
           'sebastiano', 'bertrand', 'kevin',
           'felix', 'benoit', 'arnaud']

for p in persons:
    if p == 'danielle':
        database[p] = fr_utils.img_to_encoding(os.path.join('images', "{}.png".format(p)), FRmodel)
    else:
        database[p] = fr_utils.img_to_encoding(os.path.join('images', "{}.jpg".format(p)), FRmodel)

In [11]:
def verify(image_path, identity, database, model):
    """
    对“identity”与“image_path”的编码进行验证。
    Args:
        image_path: 摄像头的图片。
        identity: 字符类型，想要验证的人的名字。
        database: 字典类型，包含了成员的名字信息与对应的编码。
        model:在Keras的模型的实例。

    Returns:
        dist -- 摄像头的图片与数据库中的图片的编码的差距。
        is_open_door -- boolean,是否该开门。
    """
    #step 1 ：计算图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = fr_utils.img_to_encoding(image_path, model)

    #step 2 ：计算与数据库中保存的编码的差距
    dist = np.linalg.norm(encoding - database[identity])

    #step 3 ：判断是否打开门
    if dist < 0.7:
        print("欢迎{}回家！".format(identity))
        is_open_door = True
    else:
        print("经验证，您不是{}!".format(identity))
        is_open_door = False
    return dist, is_open_door

In [12]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

欢迎younes回家！


(0.66714, True)

In [21]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

经验证，您不是kian!


(0.8586887, False)

In [22]:
#### 3.2 - 人脸识别

In [23]:
def who_is_it(image_path, database,model):
    """
    根据指定的图片来进行人脸识别
    Args:
        image_path: 图像地址
        database: 包含了名字与编码的字典
        model: 在Keras中的模型的实例。

    Returns:
        min_dist -- 在数据库中与指定图像最相近的编码。
        identity -- 字符串类型，与min_dist编码相对应的名字。
    """
    #step 1：计算指定图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = fr_utils.img_to_encoding(image_path, model)

    #step 2 ：找到最相近的编码
    ## 初始化min_dist变量为足够大的数字，这里设置为100
    min_dist = 100

    ## 遍历数据库找到最相近的编码
    for (name, db_enc) in database.items():
        ### 计算目标编码与当前数据库编码之间的L2差距。
        dist = np.linalg.norm(encoding - db_enc)

        ### 如果差距小于min_dist，那么就更新名字与编码到identity与min_dist中。
        if dist < min_dist:
            min_dist = dist
            identity = name

    # 判断是否在数据库中
    if min_dist > 0.7:
        print("抱歉，您的信息不在数据库中。")

    else:
        print("姓名 {}  差距：{}".format(identity, min_dist))

    return min_dist, identity

In [24]:
who_is_it("images/camera_0.jpg", database, FRmodel)

姓名 younes  差距：0.667140007019043


(0.66714, 'younes')